# Running On-Prem SQL DB Queries via FCP
Demonstrate how to get aggregate info or import cohorts from queries run on SQL databases that are located on-prem

#### Prerequisites 
1. Have a SQL DB that is open to connections from a Rhino Client, with access credentials for read-only access to this DB
2. Have a project where you are either part of the project's lead workgroup and the DB is within your site, or where there is a collaborator in the project that has the DB at their site
3. Ensure the required site-level permissions for SQL querying (Import and export cohorts, View cohort analytics) are enabled for the site that has the DB

### 1. Initialization and Login

In [ ]:
import getpass
from pprint import pprint

import rhino_health as rh
from rhino_health.lib.endpoints.sql_query.sql_query_dataclass import (
    SQLQueryImportInput,
    SQLQueryInput,
    SQLServerTypes,
    ConnectionDetails,
)
from rhino_health.lib.metrics import Count, FilterType, Mean, StandardDeviation

In [ ]:
print("Logging In")
my_username = "my_email@example.com" # Replace this with the email you use to log into Rhino Health
my_password = getpass.getpass()
session = rh.login(username=my_username, password=my_password)
print("Logged In")

### 2. Connection Setup

When specifying the connection details, ensure that you provide the server_type using the approved SQLServerTypes enum. This step ensures that your server is supported and compatible with the querying process.

In [ ]:
sql_db_user = "" # Replace this with your DB username (make sure the user has read-only permissions to the DB).
sql_db_password = "" # Replace this with your DB password. For better security, consider using an environment variable, e.g. - os.getenv("DB_PASSWORD")., or using getpass.getpass() to type in the password
external_server_url = "" # Replace this with url + port of the SQL DB you want to query (ie "{url}:{port}").
db_name = "" # Replace this with your DB name.

connection_details = ConnectionDetails(
    server_user=sql_db_user,
    password=sql_db_password,    
    server_type=SQLServerTypes.POSTGRESQL, # Replace POSTGRESQL with the relevant type of your sql server (See docs for all supported types).
    server_url=external_server_url,
    db_name=db_name
)

#### Collect all necessary parameter for queries:

Note - You can copy your project UID from the projects page in the Rhino Health dashboard (GUI) under the 3-dot menu

In [ ]:
project_uid = session.project.get_project_by_name('Your project name').uid
# Set the workgroup_uid to the workgroup the DB belongs to.
workgroup_uid = session.project.get_collaborating_workgroups(project_uid)[0].uid

### 3. Running exploratory queries:
You can run SQL queries on the remote DB and receive aggregate statistics on the results of the query.

This involves two inputs:
1. Define the query you want to run (note that the RHP does not limit the SQL code that is run - always connect with a DB user that has read-only permissions)
2. Define the metrics you would like to calculate on the query results (using metrics in from rhino_health.lib.metrics)


In [ ]:
starting_query = "" # Replace with query you want to run, e.g. "SELECT * FROM <your_table> WHERE <condition>"
metric_definitions = [] # Define a list of metrics, e.g. [Mean(variable="Height")] (see documentation on how to define metrics in the RHP SDK)

#### Define the query run parameters

In [ ]:
query_run_params = SQLQueryInput(
    session=session,
    project=project_uid, # The project/workgroup will be used to validate permissions (including and k_anonymization value)
    workgroup=workgroup_uid,
    connection_details=connection_details,
    sql_query=starting_query,
    timeout_seconds=600,
    metric_definitions=metric_definitions
)

#### Run the query

In [ ]:
# Run the query on your SQL DB and get the metric results
response = session.sql_query.run_sql_query(query_run_params)

In [ ]:
pprint(f"The result of running the metric on the query: {response.dict()}")

### 4. Import query results as a cohort in the Rhino Health Platform
You can run SQL queries on the remote DB and then have the results of the query stored on the Rhino Client as a cohort, available for further processing, analysis, etc.

This involves two inputs:
1. The query you want to run (note that the RHP does not limit the SQL code that is run - always connect with a DB user that has read-only permissions)
2. Data needed for the cohort creation (e.g. cohort name)

In [ ]:
query = "" # Replace with query you want to run to generate the data for the cohort, e.g. "SELECT * FROM <your_table> WHERE <condition>"
cohort_name = "my_cohort" # Define cohort name 
is_data_deidentified = False # True/False - whether the data queried is deidentifed 

# If you want the schema to be automatically generated, leave data_schema_uid as None
dataschemas = session.project.get_data_schemas(project_uid)
dataschema = data_schemas[0] # Choose relevant dataschema
print(f"Loaded dataschema '{dataschema.name}' with uid '{dataschema.uid}'")

#### Define the cohort import parameters

In [ ]:
import_run_params = SQLQueryImportInput(
    session=session,
    project=project_uid, # The project/workgroup will be used to validate permissions (including and k_anonymization value)
    workgroup=workgroup_uid,
    connection_details=connection_details,
    cohort_name=cohort_name,
    data_schema_uid=dataschema.uid
    timeout_seconds=600,
    is_data_deidentified=is_data_deidentified,
    sql_query=query,
)


#### Trigger the query and cohort import

In [ ]:
# Run the query on your SQL DB and import the results as a cohort
response = session.sql_query.import_cohort_from_sql_query(import_run_params)

In [ ]:
pprint(f"The result of importing the cohort: {response.dict()}")
